In [2]:
#!pip install keras

    100% |################################| 317kB 3.2MB/s ta 0:00:011
    100% |################################| 276kB 4.4MB/s eta 0:00:01
    100% |################################| 51kB 12.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
Successfully built pyyaml
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
from pathlib import Path

import tensorflow as tf
tf_session = tf.Session()
from keras import backend as K
K.set_session(tf_session)

from keras.callbacks import ModelCheckpoint
from keras.layers import Add, Dense, Input, LSTM
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

import numpy as np
import pandas as pd


# Load Input

In [2]:
root_path = Path('../..')
input_path = root_path / 'input'
poem_path = input_path / 'poems'
haiku_path = poem_path / 'haikus.csv'

In [4]:
df = pd.read_csv(str(haiku_path))
df

,0,1,2,source,0_syllables,1_syllables,2_syllables
0,Memorial Day --,a shadow for each,white cross,tempslibres,5,5,2
1,spring rain -,as the doctor speaks,i think of lilacs,tempslibres,"2,3",5,5
2,spring moonset --,a rice ball for,breakfast,tempslibres,"3,4",4,2
3,sunny afternoon,an old man lingers,near the mailbox,tempslibres,5,5,4
4,cinco de mayo,horses roll,in the shallows,tempslibres,5,3,4
5,quitting time,the smell of rain,in the lobby,tempslibres,3,4,4
6,waves,slowly cresting towards shore,a faint moon,tempslibres,1,"6,7",3
7,overnight rain --,the scent of orange blossoms,in a desert town,tempslibres,4,7,5
8,misty summer rain,calling pheasant,in Zen temple,tempslibres,5,4,4
9,day is done,poppies amidst,the dying grass,tempslibres,3,4,4


# Format Input for Training

In [5]:
# Duplicate lines with ambiguous syllable counts
# (syllable counts where there is a comma because
# multiple pronounciations are acceptable)

lines = set([0, 1, 2])

for i in range(3):
    lines.remove(i)
    df = df[[
        '0', '1', '2',
        #'1_syllables', '2_syllables'
    ] + ['%s_syllables' % j for j in lines]].join(
        df['%s_syllables' % i].str.split(
            ',', expand=True
        ).stack(-1).reset_index(
            level=1, drop=True
        ).rename('%s_syllables' % i)
    ).drop_duplicates()
    lines.add(i)

df

,0,1,2,0_syllables,1_syllables,2_syllables
0,Memorial Day --,a shadow for each,white cross,5,5,2
1,spring rain -,as the doctor speaks,i think of lilacs,2,5,5
1,spring rain -,as the doctor speaks,i think of lilacs,3,5,5
2,spring moonset --,a rice ball for,breakfast,3,4,2
2,spring moonset --,a rice ball for,breakfast,4,4,2
3,sunny afternoon,an old man lingers,near the mailbox,5,5,4
4,cinco de mayo,horses roll,in the shallows,5,3,4
5,quitting time,the smell of rain,in the lobby,3,4,4
6,waves,slowly cresting towards shore,a faint moon,1,6,3
6,waves,slowly cresting towards shore,a faint moon,1,7,3


In [6]:
# Drop samples that are longer that the 99th percentile of length

max_line_len = int(max([df['%s' % i].str.len().quantile(.99) for i in range(3)]))
df = df[
    (df['0'].str.len() <= max_line_len) & 
    (df['1'].str.len() <= max_line_len) & 
    (df['2'].str.len() <= max_line_len)
].copy()
df

,0,1,2,0_syllables,1_syllables,2_syllables
0,Memorial Day --,a shadow for each,white cross,5,5,2
1,spring rain -,as the doctor speaks,i think of lilacs,2,5,5
1,spring rain -,as the doctor speaks,i think of lilacs,3,5,5
2,spring moonset --,a rice ball for,breakfast,3,4,2
2,spring moonset --,a rice ball for,breakfast,4,4,2
3,sunny afternoon,an old man lingers,near the mailbox,5,5,4
4,cinco de mayo,horses roll,in the shallows,5,3,4
5,quitting time,the smell of rain,in the lobby,3,4,4
6,waves,slowly cresting towards shore,a faint moon,1,6,3
6,waves,slowly cresting towards shore,a faint moon,1,7,3


In [7]:
# Pad the lines to the max line length with new lines
for i in range(3):
    # For input, duplicate the first character
    # TODO - Why?
    df['%s_in' % i] = (df[str(i)].str[0] + df[str(i)]).str.pad(max_line_len+2, 'right', '\n')
    
    # 
    #df['%s_out' % i] = df[str(i)].str.pad(max_line_len, 'right', '\n') + ('\n' if i == 2 else df[str(i+1)].str[0])
    
    # TODO - trying to add the next line's first character before the line breaks
    if i == 2: # If it's the last line
        df['%s_out' % i] = df[str(i)].str.pad(max_line_len+2, 'right', '\n')
    else: 
        # If it's the first or second line, add the first character of the next line to the end of this line.
        # This helps with training so that the next RNN has a better chance of getting the first character right.
        df['%s_out' % i] = (df[str(i)] + '\n' + df[str(i+1)].str[0]).str.pad(max_line_len+2, 'right', '\n')
    
max_line_len += 2

df

,0,1,2,0_syllables,1_syllables,2_syllables,0_in,0_out,1_in,1_out,2_in,2_out
0,Memorial Day --,a shadow for each,white cross,5,5,2,MMemorial Day --\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Memorial Day --\na\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,aa shadow for each\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,a shadow for each\nw\n\n\n\n\n\n\n\n\n\n\n\n\n...,wwhite cross\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,white cross\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
1,spring rain -,as the doctor speaks,i think of lilacs,2,5,5,sspring rain -\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring rain -\na\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,aas the doctor speaks\n\n\n\n\n\n\n\n\n\n\n\n\...,as the doctor speaks\ni\n\n\n\n\n\n\n\n\n\n\n\...,ii think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,i think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
1,spring rain -,as the doctor speaks,i think of lilacs,3,5,5,sspring rain -\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring rain -\na\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,aas the doctor speaks\n\n\n\n\n\n\n\n\n\n\n\n\...,as the doctor speaks\ni\n\n\n\n\n\n\n\n\n\n\n\...,ii think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,i think of lilacs\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
2,spring moonset --,a rice ball for,breakfast,3,4,2,sspring moonset --\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring moonset --\na\n\n\n\n\n\n\n\n\n\n\n\n\n...,aa rice ball for\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,a rice ball for\nb\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,bbreakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,breakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
2,spring moonset --,a rice ball for,breakfast,4,4,2,sspring moonset --\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,spring moonset --\na\n\n\n\n\n\n\n\n\n\n\n\n\n...,aa rice ball for\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,a rice ball for\nb\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,bbreakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,breakfast\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
3,sunny afternoon,an old man lingers,near the mailbox,5,5,4,ssunny afternoon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,sunny afternoon\na\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,aan old man lingers\n\n\n\n\n\n\n\n\n\n\n\n\n\...,an old man lingers\nn\n\n\n\n\n\n\n\n\n\n\n\n\...,nnear the mailbox\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,near the mailbox\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,cinco de mayo,horses roll,in the shallows,5,3,4,ccinco de mayo\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,cinco de mayo\nh\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,hhorses roll\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,horses roll\ni\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,iin the shallows\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,in the shallows\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
5,quitting time,the smell of rain,in the lobby,3,4,4,qquitting time\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,quitting time\nt\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,tthe smell of rain\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,the smell of rain\ni\n\n\n\n\n\n\n\n\n\n\n\n\n...,iin the lobby\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,in the lobby\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
6,waves,slowly cresting towards shore,a faint moon,1,6,3,wwaves\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,waves\ns\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,sslowly cresting towards shore\n\n\n\n\n\n\n\n...,slowly cresting towards shore\na\n\n\n\n\n\n\n...,aa faint moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,a faint moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
6,waves,slowly cresting towards shore,a faint moon,1,7,3,wwaves\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,waves\ns\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,sslowly cresting towards shore\n\n\n\n\n\n\n\n...,slowly cresting towards shore\na\n\n\n\n\n\n\n...,aa faint moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,a faint moon\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [8]:
inputs = df[['0_in', '1_in', '2_in']].values

t = Tokenizer(filters='', char_level=True)
t.fit_on_texts(inputs.flatten())
n_tokens = len(t.word_counts) + 1

# X is the input for each line in sequences of one-hot-encoded values
X = np_utils.to_categorical([
    t.texts_to_sequences(inputs[:,i]) for i in range(3)
], num_classes=n_tokens)

outputs = df[['0_out', '1_out', '2_out']].values

# Y is the output for each line in sequences of one-hot-encoded values
Y = np_utils.to_categorical([
    t.texts_to_sequences(outputs[:,i]) for i in range(3)
], num_classes=n_tokens)

# X_syllables is the count of syllables for each line
X_syllables = df[['0_syllables', '1_syllables', '2_syllables']].values

# Training Model

In [11]:
LATENT_DIM = 32
class TrainingLine:
    def __init__(self, name, previous_line, lstm):
        self.char_input = Input(shape=(None, n_tokens), name='char_input_%s' % name)
        
        self.syllable_input = Input(shape=(1,), name='syllable_input_%s' % name)
        self.syllable_dense = Dense(lstm.units, activation='relu', name='syllable_dense_%s' % name)
        self.syllable_dense_output = self.syllable_dense(self.syllable_input)
        
        #self.lstm = LSTM(latent_dim, return_state=True, return_sequences=True, name='lstm_%s' % name)
        
        if previous_line:
            initial_state = [
                Add(name='add_h_%s' % name)([
                    previous_line.lstm_h,
                    self.syllable_dense_output
                ]),
                Add(name='add_c_%s' % name)([
                    previous_line.lstm_c,
                    self.syllable_dense_output
                ])
            ]
        else:
            initial_state = [self.syllable_dense_output, self.syllable_dense_output]
            
        self.lstm_out, self.lstm_h, self.lstm_c = lstm(self.char_input, initial_state=initial_state)
        
        self.output_dense = Dense(n_tokens, activation='softmax', name='output_%s' % name)
        self.output = self.output_dense(self.lstm_out)
        
lstm = LSTM(LATENT_DIM, return_state=True, return_sequences=True, name='lstm')
lines = []
inputs = []
outputs = []
for i in range(3):
    previous_line = lines[-1] if lines else None
    lines.append(TrainingLine('line_%s' % i, previous_line, lstm))
    inputs += [lines[-1].char_input, lines[-1].syllable_input]
    outputs.append(lines[-1].output)
    
training_model = Model(inputs, outputs)
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
training_model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
syllable_input_line_0 (InputLayer)                                (None, 1)                                   0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
char_input_line_0 (InputLayer)                                    (None, None, 83)                            0                                                                                     

In [13]:
# Either train the model or load an existing model

filepath="model-6-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

training_model.fit([
    X[0], X_syllables[:,0], 
    X[1], X_syllables[:,1], 
    X[2], X_syllables[:,2]
], [Y[0], Y[1], Y[2]], batch_size=64, epochs=1000, validation_split=.1, callbacks=callbacks_list)

#training_model.save('new_model_5.hdf5')

#training_model.load_weights('new_model_4.hdf5')

Train on 33316 samples, validate on 3702 samples
Epoch 1/1000
33316/33316 [==============================] - 123s 4ms/step - loss: 3.0510 - output_line_0_loss: 0.9724 - output_line_1_loss: 1.1201 - output_line_2_loss: 0.9585 - val_loss: 3.1372 - val_output_line_0_loss: 0.9335 - val_output_line_1_loss: 1.2912 - val_output_line_2_loss: 0.9126

Epoch 00001: val_loss improved from -inf to 3.13724, saving model to model-6-01-3.14.hdf5


/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'syllable_dense_line_0_1/Relu:0' shape=(?, 32) dtype=float32>, <tf.Tensor 'syllable_dense_line_0_1/Relu:0' shape=(?, 32) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'add_h_line_1_1/add:0' shape=(?, 32) dtype=float32>, <tf.Tensor 'add_c_line_1_1/add:0' shape=(?, 32) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/keras/engine/netw

Epoch 2/1000
33316/33316 [==============================] - 121s 4ms/step - loss: 2.6106 - output_line_0_loss: 0.8396 - output_line_1_loss: 0.9598 - output_line_2_loss: 0.8112 - val_loss: 2.7344 - val_output_line_0_loss: 0.8129 - val_output_line_1_loss: 1.1209 - val_output_line_2_loss: 0.8007

Epoch 00002: val_loss did not improve from 3.13724
Epoch 3/1000
33316/33316 [==============================] - 125s 4ms/step - loss: 2.3708 - output_line_0_loss: 0.7597 - output_line_1_loss: 0.8700 - output_line_2_loss: 0.7412 - val_loss: 2.5697 - val_output_line_0_loss: 0.7639 - val_output_line_1_loss: 1.0550 - val_output_line_2_loss: 0.7508

Epoch 00003: val_loss did not improve from 3.13724
Epoch 4/1000
33316/33316 [==============================] - 127s 4ms/step - loss: 2.2459 - output_line_0_loss: 0.7167 - output_line_1_loss: 0.8263 - output_line_2_loss: 0.7028 - val_loss: 2.4555 - val_output_line_0_loss: 0.7297 - val_output_line_1_loss: 1.0108 - val_output_line_2_loss: 0.7150

Epoch 00004: 

KeyboardInterrupt: 

In [17]:
import gc
del generator_lines
gc.collect()

0

In [19]:
lstm.get_weights()

[array([[-9.0790652e-03,  8.2711186e-03, -2.6545394e-03, ...,
         -6.6156350e-03, -3.4619737e-03, -7.6799411e-03],
        [-2.3324497e-02, -1.4255616e-02, -7.0373690e-01, ...,
         -5.7577354e-01, -1.1094084e-01,  3.4616303e-01],
        [-3.0286992e-01, -2.0568984e-02, -5.0969672e-01, ...,
         -3.4649441e-01, -3.8086605e-01, -4.2206904e-01],
        ...,
        [-1.6359581e-02,  1.9466395e-02,  4.7066985e-03, ...,
         -3.9402939e-02, -8.7300371e-03,  2.9377444e-02],
        [-1.1438133e-02,  6.4846594e-03,  2.2234170e-02, ...,
         -7.8754388e-03, -2.0130802e-02, -1.5592977e-02],
        [ 2.4709124e-03,  1.3276562e-04, -1.3405377e-02, ...,
          9.6113253e-03, -1.0683596e-02, -2.6530089e-02]], dtype=float32),
 array([[ 0.08603223,  0.06558695,  0.08990103, ..., -0.07968667,
          0.05276636,  0.04927232],
        [ 0.11359484, -0.14226615,  0.09356155, ...,  0.02646664,
         -0.09922902,  0.32840014],
        [-0.20365952, -0.01859319, -0.20309702

In [21]:
generator_lstm = LSTM(LATENT_DIM, return_state=True, return_sequences=True, name='generator_lstm')

class GeneratorLine:
    def __init__(self, name, training_line, lstm, latent_dim=LATENT_DIM):
        self.char_input = Input(shape=(None, n_tokens), name='char_input_%s' % name)
        
        self.syllable_input = Input(shape=(1,), name='syllable_input_%s' % name)
        self.syllable_dense = Dense(latent_dim, activation='relu', name='syllable_dense_%s' % name)
        self.syllable_dense_output = self.syllable_dense(self.syllable_input)
        
        self.h_input = Input(shape=(latent_dim,), name='h_input_%s' % name)
        self.c_input = Input(shape=(latent_dim,), name='c_input_%s' % name)
        initial_state = [self.h_input, self.c_input]
        
        self.lstm = lstm
            
        self.lstm_out, self.lstm_h, self.lstm_c = self.lstm(self.char_input, initial_state=initial_state)
        
        self.output_dense = Dense(n_tokens, activation='softmax', name='output_%s' % name)
        self.output = self.output_dense(self.lstm_out)
        
        self.syllable_dense.set_weights(training_line.syllable_dense.get_weights())
        #self.lstm.set_weights(lstm.get_weights())
        self.output_dense.set_weights(training_line.output_dense.get_weights())

generator_lines = [GeneratorLine('generator_line_%s' % i, lines[i], generator_lstm) for i in range(3)]        
generator_lstm.set_weights(lstm.get_weights())

In [29]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [93]:
for _ in range(10):
    syllables = [5, 7, 5]
    h = None
    c = None
    #next_char = 4
    next_char = t.texts_to_sequences(chr(int(np.random.randint(ord('a'), ord('z')+1))))[0][0]

    for i in range(3):
        l = generator_lines[i]

        s = tf_session.run(
            l.syllable_dense_output,
            feed_dict={
                l.syllable_input: [[syllables[i]]]
            }
        )

        if h is None:
            h = s
            c = s
        else:
            h = h + s
            c = c + s

        line = [next_char]

        end = False
        next_char = None
        for i in range(max_line_len):
            char, h, c = tf_session.run(
                [l.output, l.lstm_h, l.lstm_c],
                feed_dict={
                    l.char_input: [[np_utils.to_categorical(line[-1], num_classes=n_tokens)]],
                    l.h_input: h,
                    l.c_input: c
                }
            )

            #char = np.argmax(char)
            char = sample(char[0,0], .4)
            if char == 1 and not end:
                end = True
            if char != 1 and end:
                next_char = char
                char = 1

            line.append(char)

        print(t.sequences_to_texts([line])[0].strip()[1:].replace('   ', '\n').replace(' ', '').replace('\n', ' '))
    print()

just before darkness
fireflies rise fresh and dreams
was in the footprints

family christmas -
she longs out for her children
to the starry pies

x we have always
and out of the summer grass
and worshipped me and me

between the second
and the world shall revere us
and enters the sur

family christmas -
she pray for me giving up
to hear the crass red

before thy thermal
and traul cans in the stone
's cold might save the

family christmas -
she leaves her good intentions
in a chorpy'e book

a child is venged closed
the charge lives now spoken sick here
is all help says

before thy thermal
and all that's best of darkness
your whate forevermors

between the second
and when the only seeming
mountains of giants



In [91]:
word = 'today is celebrating'
df[df['0'].str.contains(word) | df['1'].str.contains(word) | df['2'].str.contains(word) ]

,0,1,2,0_syllables,1_syllables,2_syllables,0_in,0_out,1_in,1_out,2_in,2_out
15708,every place every thing every house today is...,today everything is bright cheerful a chimera,today is the feast of spring,16,13,8,eevery place every thing every house today i...,every place every thing every house today is...,ttoday everything is bright cheerful a chime...,today everything is bright cheerful a chimer...,ttoday is the feast of spring\n\n\n\n\n\n\n\n\...,today is the feast of spring\n\n\n\n\n\n\n\n\n...
15708,every place every thing every house today is...,today everything is bright cheerful a chimera,today is the feast of spring,16,13,7,eevery place every thing every house today i...,every place every thing every house today is...,ttoday everything is bright cheerful a chime...,today everything is bright cheerful a chimer...,ttoday is the feast of spring\n\n\n\n\n\n\n\n\...,today is the feast of spring\n\n\n\n\n\n\n\n\n...
15708,every place every thing every house today is...,today everything is bright cheerful a chimera,today is the feast of spring,17,13,8,eevery place every thing every house today i...,every place every thing every house today is...,ttoday everything is bright cheerful a chime...,today everything is bright cheerful a chimer...,ttoday is the feast of spring\n\n\n\n\n\n\n\n\...,today is the feast of spring\n\n\n\n\n\n\n\n\n...
15708,every place every thing every house today is...,today everything is bright cheerful a chimera,today is the feast of spring,17,13,7,eevery place every thing every house today i...,every place every thing every house today is...,ttoday everything is bright cheerful a chime...,today everything is bright cheerful a chimer...,ttoday is the feast of spring\n\n\n\n\n\n\n\n\...,today is the feast of spring\n\n\n\n\n\n\n\n\n...
15708,every place every thing every house today is...,today everything is bright cheerful a chimera,today is the feast of spring,18,13,8,eevery place every thing every house today i...,every place every thing every house today is...,ttoday everything is bright cheerful a chime...,today everything is bright cheerful a chimer...,ttoday is the feast of spring\n\n\n\n\n\n\n\n\...,today is the feast of spring\n\n\n\n\n\n\n\n\n...
15708,every place every thing every house today is...,today everything is bright cheerful a chimera,today is the feast of spring,18,13,7,eevery place every thing every house today i...,every place every thing every house today is...,ttoday everything is bright cheerful a chime...,today everything is bright cheerful a chimer...,ttoday is the feast of spring\n\n\n\n\n\n\n\n\...,today is the feast of spring\n\n\n\n\n\n\n\n\n...
15708,every place every thing every house today is...,today everything is bright cheerful a chimera,today is the feast of spring,19,13,8,eevery place every thing every house today i...,every place every thing every house today is...,ttoday everything is bright cheerful a chime...,today everything is bright cheerful a chimer...,ttoday is the feast of spring\n\n\n\n\n\n\n\n\...,today is the feast of spring\n\n\n\n\n\n\n\n\n...
15708,every place every thing every house today is...,today everything is bright cheerful a chimera,today is the feast of spring,19,13,7,eevery place every thing every house today i...,every place every thing every house today is...,ttoday everything is bright cheerful a chime...,today everything is bright cheerful a chimer...,ttoday is the feast of spring\n\n\n\n\n\n\n\n\...,today is the feast of spring\n\n\n\n\n\n\n\n\n...


In [43]:
t.texts_to_sequences(['a', 'b', 'c'])

[[5], [21], [19]]

In [44]:
t.index_word

{1: '\n',
 2: ' ',
 3: 'e',
 4: 't',
 5: 'a',
 6: 'o',
 7: 's',
 8: 'n',
 9: 'i',
 10: 'r',
 11: 'h',
 12: 'l',
 13: 'd',
 14: 'u',
 15: 'm',
 16: 'w',
 17: 'f',
 18: 'g',
 19: 'c',
 20: 'y',
 21: 'b',
 22: 'p',
 23: 'k',
 24: 'v',
 25: '-',
 26: "'",
 27: '.',
 28: 'j',
 29: 'z',
 30: 'x',
 31: 'q',
 32: '?',
 33: ':',
 34: ',',
 35: '[',
 36: '\x97',
 37: '`',
 38: '_',
 39: '(',
 40: ')',
 41: '1',
 42: '"',
 43: '~',
 44: '0',
 45: '!',
 46: '2',
 47: ';',
 48: '\x92',
 49: '>',
 50: '3',
 51: '<',
 52: '/',
 53: '7',
 54: '8',
 55: '&',
 56: '4',
 57: '\xa0',
 58: '\x96',
 59: '6',
 60: 'é',
 61: '5',
 62: '9',
 63: ']',
 64: '\x85',
 65: '*',
 66: '\u200b',
 67: '=',
 68: '{',
 69: 'ä',
 70: '@',
 71: '%',
 72: '…',
 73: 'ê'}

In [54]:
np.random.randint(ord('a'), ord('z')+1)#, 100).max()

118

In [53]:
ord('z')

122

In [57]:
chr(int(np.random.randint(ord('a'), ord('z')+1)))

'e'

In [59]:
t.texts_to_sequences(chr(int(np.random.randint(ord('a'), ord('z')+1))))[0]

[29]